In [1]:
#import shapX as shap
import transformers
import torch
import datasets
import yaml
import numpy as np
import scipy as sp
import pandas as pd
import shap
from tqdm import tqdm
import numpy as np
import pandas as pd
from datetime import datetime
import datasets
import os
import scipy as sp
import gc
import time
import torch
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import re
import unicodedata
import yaml
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, BatchEncoding, Trainer, TrainingArguments, AdamW, get_scheduler
from src.utils.utils import fetch_import_module
from pipelines import utils_pipeline
from time import gmtime, strftime
from tqdm import tqdm
from datasets import concatenate_datasets,Dataset,ClassLabel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import sys
from torch.utils.data import Dataset, TensorDataset, DataLoader
import subprocess
from accelerate import Accelerator
os.environ['MKL_THREADING_LAYER'] = 'GNU'
from sklearn.model_selection import train_test_split
from pathlib import Path
from transformers import BertModel
from torch import nn

2022-10-01 19:44:26.734603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 19:44:26.889331: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 19:44:26.889356: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-01 19:44:26.920321: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-01 19:44:27.558781: W tensorflow/stream_executor/platform/de

In [2]:
twitter_username_re = re.compile(r'@([A-Za-z0-9_]+)')
hashtag_re = re.compile(r'\B(\#[a-zA-Z0-9]+\b)(?!;)')
html_symbol_re = re.compile(r'&[^ ]+')
lbr_re = re.compile(r'\|LBR\|')
url_re = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')


In [3]:
def calculateError(x, dataset_names):
    error = 0
    for dataset in dataset_names:
        if x['Labels'] != x[dataset]:
            error += 1
    return error

def cleanTweets(tweet):
    text = twitter_username_re.sub("[UNK]",tweet)
    text = lbr_re.sub(' ', text)
    text = unicodedata.normalize('NFKC',text)
    text = text.replace('\n',' ')
    text = text.replace('RT ',' ')
    text = hashtag_re.sub("[UNK]",text)
    text = html_symbol_re.sub(" ",text)
    text = url_re.sub("[UNK]",text)
    return text

def predict(x):
    input_ids = []
    attention_masks = []
    for record in x:
        processed_x = preprocess(record, tokenizer)
        input_ids.append(processed_x["input_ids"])
        attention_masks.append(processed_x["attention_mask"])

    input_ids_tensor = torch.cat(input_ids)
    attention_masks_tensor = torch.cat(attention_masks)

    input = torch.stack((input_ids_tensor, attention_masks_tensor))
    reshaped_input = torch.reshape(input, (-1, 2, 512)).to(device)
    
    output = torch.sigmoid(model(reshaped_input)).cpu().detach().numpy()

    return output

def preprocess(batch, tokenizer):
        batch = cleanTweets(batch)
        truncation_length = 512
        return tokenizer(batch, truncation=True, max_length=truncation_length, padding = "max_length" ,return_token_type_ids = False, return_tensors="pt")

In [4]:
dataset_names = [
        "germeval2018", 
        "germeval2019",
        "hasoc2019",
        "hasoc2020",
        "ihs_labelled",
        "covid_2021",
        "telegram_gold"
    ]
n = 10

In [5]:
# tokenizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = transformers.AutoTokenizer.from_pretrained('deepset/gbert-base')

In [6]:
abusive_df_list = []
neutral_df_list = []

for dataset in dataset_names:
    # fetch datasets
    label2id = {"neutral": 0, "abusive":1}
    # import dataset pipeline
    source_module = fetch_import_module(dataset)
    # execute get_data_binary in pipeline
    dset_list_of_dicts = source_module.get_data_binary()
    # convert list to dataframe
    dset_df = pd.DataFrame(dset_list_of_dicts)

    #map neutral to 0 and abusive to 1
    dset_df["label"] = dset_df["label"].map(label2id)

    neutral_df = dset_df[dset_df["label"] == 0][:n]
    abusive_df = dset_df[dset_df["label"] == 1][:n]

    abusive_df_list.append(abusive_df)
    neutral_df_list.append(neutral_df)

combined_abusive_df = pd.concat(abusive_df_list)
combined_neutral_df = pd.concat(neutral_df_list)

combined_dataset = pd.concat([combined_abusive_df, combined_neutral_df])["text"].to_list()

In [8]:
class SHAP_BERT_cnn(nn.Module):
    def __init__(self, bottleneck_dim = 768):
        super(SHAP_BERT_cnn, self).__init__()
        # truncation length = 512
        self.feature_extractor_conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding = (1,0))
        self.feature_extractor_relu = nn.ReLU()
        self.feature_extractor_pool = nn.MaxPool1d(kernel_size=3)
        self.feature_extractor_fc = nn.Linear(2210, bottleneck_dim)
        self.feature_extractor_dropout = nn.Dropout(0.1)
        self.feature_extractor_flat = nn.Flatten()
       
    def forward(self, bert_output, input_is_bert = True):
        if input_is_bert:
            x = torch.swapaxes(torch.stack(bert_output[2]), 0, 1)
        else:
            x = bert_output
        x = self.feature_extractor_dropout(x)

        x = self.feature_extractor_conv(x)
        x = x.squeeze()
        x = self.feature_extractor_relu(x)
        x = self.feature_extractor_dropout(x)
        x = self.feature_extractor_pool(x)
        x = self.feature_extractor_dropout(x)
        if x.size(dim=0) == 13 and x.size(dim=1) == 170:
            x = x.unsqueeze(0)
        x = self.feature_extractor_flat(x)
        x = self.feature_extractor_dropout(x)
        x = self.feature_extractor_fc(x)
        return x

class DANN_task_classifier(nn.Module):
    def __init__(self, bottleneck_dim = 768, layer_size = 100):
        super().__init__()

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(bottleneck_dim, layer_size))
        self.class_classifier.add_module('c_relu1', nn.ReLU())
        self.class_classifier.add_module('c_drop1', nn.Dropout())
        self.class_classifier.add_module('c_fc2', nn.Linear(layer_size, layer_size))
        self.class_classifier.add_module('c_relu2', nn.ReLU())
        self.class_classifier.add_module('c_fc3', nn.Linear(layer_size, 1))
        self.class_classifier.add_module('c_flatten', nn.Flatten(start_dim = 0))
        

    def forward(
        self,
        feature_extractor_output
    ):
    
        x = self.class_classifier(feature_extractor_output)

        return x

class labelled_only_model(nn.Module):
    def __init__(self, feature_extractor_module, task_classifier_module):
        super(labelled_only_model, self).__init__()
        self.bert = BertModel.from_pretrained("deepset/gbert-base")
        self.output_hidden_states = True
        self.feature_extractor = feature_extractor_module
        self.task_classifier = task_classifier_module

    def forward(self, input_data):
        bert_output = self.bert(input_ids=input_data[:,0], attention_mask=input_data[:,1], return_dict = False, output_hidden_states=self.output_hidden_states)

        feature_extractor_output = self.feature_extractor(bert_output)

        class_output = self.task_classifier(feature_extractor_output)

        return class_output
    
    # inference for testing
    def inference(self, input_data):
        bert_output = self.bert(input_ids=input_data[:,0], attention_mask=input_data[:,1], return_dict = False, output_hidden_states=self.output_hidden_states)
        
        feature_extractor_output = self.feature_extractor(bert_output)

        class_output = self.task_classifier(feature_extractor_output)

        return class_output


In [9]:
# exp_name = "single_source_germeval2018_germeval2018_fair_false"
# path_state_dict = Path(f"./model_state_dicts/{exp_name}")
feature_extractor = SHAP_BERT_cnn(850)
task_classifier = DANN_task_classifier(850, 850)
model = labelled_only_model(feature_extractor, task_classifier).to(device)
# model.load_state_dict(torch.load(path_state_dict))
# model.eval()

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# def custom_tokenizer(s, return_offsets_mapping=True):
#     """ Custom tokenizers conform to a subset of the transformers API.
#     """
#     pos = 0
#     offset_ranges = []
#     input_ids = []
#     for m in re.finditer(r"\W", s):
#         start, end = m.span(0)
#         offset_ranges.append((pos, start))
#         input_ids.append(s[pos:start])
#         pos = end
#     if pos != len(s):
#         offset_ranges.append((pos, len(s)))
#         input_ids.append(s[pos:])
#     out = {}
#     out["input_ids"] = input_ids
#     if return_offsets_mapping:
#         out["offset_mapping"] = offset_ranges
#     return out

In [13]:
#masker = shap.maskers.Text(custom_tokenizer)

explainer = shap.Explainer(predict, masker=tokenizer)

shap_values = explainer(combined_dataset[:5])

2022-10-01 19:44:42.305300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-01 19:44:42.305459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 19:44:42.305519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-01 19:44:42.305585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-01 19:44:42.305686: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

  0%|          | 0/248 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
shap.plots.text(shap_values[2])